In [1]:
from battle import *
from trainer import *
from pokemon import *
from moves import *
from globals import *
from team import *
from event import *

In [2]:
moves=[Tackle(), Earthquake(), Thunder()]
pikachuStats={
    Stat.HP : 200,
    Stat.ATT : 100,
    Stat.DEF : 100,
    Stat.SPA : 100,
    Stat.SPD : 100,
    Stat.SPE : 100
}
pikachus=[Pikachu('pika1', 15, pikachuStats, moves), Pikachu('pika2', 15, pikachuStats, moves), Pikachu('pika3', 15, pikachuStats, moves), Pikachu('pika4', 15, pikachuStats, moves)]

ratStats={
    Stat.HP : 200,
    Stat.ATT : 100,
    Stat.DEF : 100,
    Stat.SPA : 100,
    Stat.SPD : 100,
    Stat.SPE : 101
}
rats=[Rattata('rat1', 20, ratStats, moves, Sword()), Rattata('rat2', 20, ratStats, moves), Rattata('rat3', 20, ratStats, moves), Rattata('rat4', 20, ratStats, moves)]
t1=Trainer('Trainer 1', pikachus, None)
t2=Trainer('Trainer 2', rats, None)
teams=[Team('Team Pikachu', [t1], 1), Team('Team Rattata', [t2], 1)]
testBattle=Battle(teams)

In [3]:
testBattle.runBattle()
# TODO: sync pokemon fainted status with corresponding slot being empty
# TODO: only show trainers when swapping and sending out pokemon if they have pokemon available

Earthquake was super effective!
A critical hit!
pika2 took 300.0 damage!
pika2 has fainted!
A critical hit!
pika3 took 45.0 damage!
rat2 took 180.0 damage!
Thunder was ineffective...
pika3 took 60.0 damage!
rat2 took 100.0 damage!
rat2 has fainted!
